## Create a file where each point is associated with an atlas index


Find the id on downsampled points, then back track to the origional points based on the indexes of these points

The, plot the origional points in 3d with its associated atlas indexes colour coded


In [1]:
import pandas as pd

import numpy as np

import SimpleITK as sitk

import warnings

import tkinter.filedialog as fdialog


In [2]:
points_name=fdialog.askopenfile( title='Select the downsampled points').name

ori_points_name=fdialog.askopenfile( title='Select the origional points').name

atlas_name=fdialog.askopenfile( title='Select the corresponding atlas').name

outdir = fdialog.askdirectory(title='Please select the output directory')


In [3]:
atlas_labels=pd.read_csv('D:\Allenbrainatlas\ARA_25_micron_mhd_ccf2017\labels.csv')

#Read the output points
output=open(points_name,'r')
outputpoint= output.readlines()

#read the atlas
atlas= sitk.ReadImage(atlas_name)

In [4]:
all_points=[]
for lines in outputpoint[2:]:
    this_line= lines.split (' ')
    mypoints= [int(stuff) for stuff in this_line]
    all_points.append(mypoints)

In [5]:
#find the corresponding atlasIDs of the points
points_in_atlas=[int(atlas[i]) for i in all_points ]

In [6]:
#Load the origional points
anno=open(ori_points_name,'r')
anno_data=anno.readlines()
# heading is stored in anno_data[2], 1st line basically useless

headings=anno_data[2].split(' ')
annotations=[lines.split(' ') for lines in anno_data[3:]]
annotation_df=pd.DataFrame(annotations, columns=headings)

annotation_df.head()

,#n,type,x,y,z,radius,parent,seg_id,level,mode,timestamp,TFresindex
0,268943,3,14013.111,2887.333,348.444,1.000,268944,0,0,1,625570213,1\n
1,268944,3,14013.667,2888.000,348.333,1.000,268945,0,0,1,625570213,1\n
2,268945,3,14014.444,2888.667,348.889,1.000,268946,0,0,1,625570213,1\n
3,268946,3,14015.667,2889.000,348.667,1.000,268947,0,0,1,625570213,1\n
4,268947,3,14017.111,2889.000,348.111,1.000,268948,0,0,1,625570213,1\n


In [7]:
# create a list where row1= x1 y1 z1 ID1 row2= x2 y2 z2 ID2 and etc 
# for writing to the text file.

# store in to list of strings
# strpts_inatlas= [str(x) for x in points_in_atlas]
# point_list = [' '.join(x) for x in zip(annotation_df['x'],annotation_df['y'],annotation_df['z'], strpts_inatlas)]
# Use the pd dataframe to export to csv file is better


In [8]:
#save these info as pd_Dataframes
points_with_id= pd.DataFrame (zip(annotation_df['x'],annotation_df['y'], annotation_df['z'],points_in_atlas ), columns=['x', 'y','z', 'atlasID'])
points_with_id.to_csv (outdir+'/oripoints_withID.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [9]:
points_with_id.head(10)

,x,y,z,atlasID
0,14013.111,2887.333,348.444,778
1,14013.667,2888.000,348.333,778
2,14014.444,2888.667,348.889,778
3,14015.667,2889.000,348.667,778
4,14017.111,2889.000,348.111,778
5,14018.667,2889.333,348.000,778
6,14021.556,2890.444,348.000,778
7,14023.000,2891.889,348.111,778
8,13996.667,2880.000,349.667,778
9,13997.333,2880.778,349.111,778


In [10]:
import plotly
import plotly.graph_objects as go
from plotly import __version__
print(__version__)



from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

4.5.0


In [11]:
#Assign a colour to each unique atlas ID for plotting
#this is need because atlas IDs has very high values which is outside of the colour range
uniqueID=np.unique(points_with_id['atlasID'])
colour= np.linspace(1,np.size(uniqueID)+1, num=np.size(uniqueID),dtype='int')

colourdict=dict(zip(uniqueID,colour))

{0: 1,
 156: 2,
 201: 3,
 252: 4,
 401: 5,
 433: 6,
 670: 7,
 678: 8,
 776: 9,
 778: 10,
 862: 11,
 940: 12,
 981: 13,
 1006: 14,
 1038: 15,
 1046: 16,
 1047: 17,
 1062: 18,
 1070: 19,
 1090: 20,
 1111: 21,
 182305696: 22,
 312782560: 23,
 312782592: 24,
 312782624: 26}

In [12]:
atlas_labels.head()

,id,atlas_id,name,acronym,st_level,ontology_id,hemisphere_id,weight,parent_structure_id,depth,...,graph_order,structure_id_path,color_hex_triplet,neuro_name_structure_id,neuro_name_structure_id_path,failed,sphinx_id,structure_name_facet,failed_facet,safe_name
0,1000,690.0,extrapyramidal fiber systems,eps,2,1,3,8690,1009.0,2,...,1218,/997/1009/1000/,CCCCCC,NaN,NaN,f,1219,738013408,734881840,extrapyramidal fiber systems
1,223,27.0,Arcuate hypothalamic nucleus,ARH,8,1,3,8690,157.0,6,...,733,/997/8/343/1129/1097/157/223/,FF5D50,NaN,NaN,f,734,218062747,734881840,Arcuate hypothalamic nucleus
2,12998,NaN,"Somatosensory areas, layer 6b",SS6b,11,1,3,8690,453.0,7,...,36,/997/8/567/688/695/315/453/12998/,188064,NaN,NaN,f,37,3017218874,734881840,Somatosensory areas layer 6b
3,163,1010.0,"Agranular insular area, posterior part, layer 2/3",AIp2/3,11,1,3,8690,111.0,8,...,287,/997/8/567/688/695/315/95/111/163/,219866,NaN,NaN,f,288,2735510231,734881840,Agranular insular area posterior part layer 2/3
4,552,917.0,"Pontine reticular nucleus, ventral part",PRNv,8,1,3,8690,987.0,6,...,914,/997/8/343/1065/771/987/552/,FFBA86,NaN,NaN,f,915,3053786086,734881840,Pontine reticular nucleus ventral part


In [13]:
namedict=dict(zip(atlas_labels['id'],atlas_labels['name']))
points_with_id['name'] = points_with_id['atlasID'].map(namedict)

In [14]:
points_with_id['colour'] = points_with_id['atlasID'].map(colourdict)
points_with_id.head()

,x,y,z,atlasID,name,colour
0,14013.111,2887.333,348.444,778,"Primary visual area, layer 5",10
1,14013.667,2888.000,348.333,778,"Primary visual area, layer 5",10
2,14014.444,2888.667,348.889,778,"Primary visual area, layer 5",10
3,14015.667,2889.000,348.667,778,"Primary visual area, layer 5",10
4,14017.111,2889.000,348.111,778,"Primary visual area, layer 5",10


In [15]:
trace= go.Scatter3d( 
    x=points_with_id['x'],
    y=points_with_id['y'],
    z=points_with_id['z'],
    mode='markers',
    marker=dict(
        size=1,
        color=points_with_id['colour'],
        opacity=0.5),
    hovertext= points_with_id['name'],
    hoverinfo='text'
)

layout=go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0)
)

#Here we want horizontal view with dorsal surface toward us
#first we set a xz view for coronal section this is done by setting x and z as 0 in eye
# then, for y we want the smaller values to point toward us (because in our original coronal images dorsal surface is pointed up)
camera = dict(
    eye=dict(x=0, y=-1, z=0)
)


#Need to change the axis title depending on the direction and orientation of the origional image (ie, imaging started from cerebellum vs. imaging started from olfactorybulb)
fig=go.Figure(data=trace,layout=layout)
fig.update_layout(scene_camera=camera, 
                  scene=dict(
                    xaxis_title='Smaller=more medial',
                    yaxis_title='Smaller=more dorsal',
                    zaxis_title='Smaller=more posterior'))
plot(fig,filename=outdir+'/'+ outdir[3:]+'3Dplot.html')

'D:/AL080/AL0803Dplot.html'